# Разработка простой слот-машины

## Описание задания

Вам нужно будет произвести расчеты для простой слот машины, и указать параметры, которые вы задали при проведении расчетов, и к каким показателям пришли.

Ваш слот работает по правилам реальных физических слотов: каждый барабан представляет собой ленту, заполненную символами. Каждый спин ленты независимо и равновероятно останавливаются в одном из своих положений. Длина одной ленты должна быть до 200 символов.


Параметры вашей слот машины:
- 8 обычных символов.
- Wild символ.
- Символ Free Spins.
- Сетка 3х3 (у слота 3 барабана, активная область барабана - 3 символа).
- 5 линий (можете взять произвольное расположение линий).
- Комбинации выплачиваются за 3 последовательных элемента на линии.
- При выпадении 3 символов фри спинов в любом месте вне зависимости от линий запустится фриспиновая игра с 10  бесплатными спинами. Во время бесплатных спинов, нельзя выиграть дополнительные бесплатные спины - используются другие ленты.
- Payout: 90 - 95%
- Ставка на игру 5 (по 1 фишке на каждую линию).
- Wild выплачивается, а также может участвовать в комбинации, заменяя собой обычные символы, но не символы Free Spins.


В качестве ответа на это задание предоставьте:
- Использованные линии.
- Ленты (барабаны) слота.
- Таблицу выплат
- Payout каждой комбинации (обычные символы и Wild) для обычной игры.
- Payout бесплатных спинов.
- Частоту срабатывания в обычной игре фриспинов.


Расчеты произведите при помощи симуляции. Вы также можете произвести проверочные расчеты аналитически. Это приветствуется и оценивается дополнительно, но не обязательно. Предоставьте код вашей симуляции и excel/google документ с параметрами для проверки.

## План работы

1. Разработаем модель слота с визуальной составляющей и виртуальным балансом игрока;

2. Подберем входные параметры слот-машины для того, чтобы добиться необходимого **payout**: 
- ленты слота;
- таблица выплат.
3. Дадим ответы на поставленные вопросы.

## Разработка модели

Установим библиотеку `emoji` - будем использовать эмодзи в качестве символов на лентах:

In [1]:
!pip install emoji
!pip install tqdm

Установим библиотеки:

In [2]:
import numpy as np
import random
import emoji
import pandas as pd
from IPython.display import Image
from tqdm import tqdm

Создадим виртуальный кошелек как класс, для удобства взаимодействия с ним:

In [3]:
class Purse():
    def __init__(self, balance):
        self.balance = balance

    def get_balance(self):
        return self.balance

    def debit(self, debited_money):
        self.balance -= debited_money 
    
    def credit(self, credited_money):
        self.balance += credited_money  

Выберем эмодзи для обозначения символов на ленте:

In [4]:
print(emoji.emojize(':star:'),
emoji.emojize(':cherries:'),
emoji.emojize(':grapes:'),
emoji.emojize(':banana:'),
emoji.emojize(':green_apple:'),
emoji.emojize(':eggplant:'),
emoji.emojize(':pineapple:'),
emoji.emojize(':watermelon:'),
emoji.emojize(':strawberry:'),
emoji.emojize(':ticket:'))

⭐ 🍒 🍇 🍌 🍏 🍆 🍍 🍉 🍓 🎫


Каждому символу будет соответствовать число от 0 до 9 включительно, где:

- 0, символ ⭐ соответствует **Wild**;

- 1, символ 🍒 ;

- 2, символ 🍇;

- 3, символ 🍌;

- 4, символ 🍏;

- 5, символ 🍆;

- 6, символ 🍍;

- 7, символ 🍉;

- 8, символ 🍓;

- 9, символ 🎫 соответствует **Free Spins**.

Зададим словарь с символами на лентах и соответствующее им число:

In [5]:
meaning_dict = {0 : emoji.emojize(':star:'), 
                1 : emoji.emojize(':cherries:'), 
                2 : emoji.emojize(':grapes:'), 
                3 : emoji.emojize(':banana:'), 
                4 : emoji.emojize(':green_apple:'), 
                5 : emoji.emojize(':eggplant:'), 
                6 : emoji.emojize(':pineapple:'), 
                7 : emoji.emojize(':watermelon:'), 
                8 : emoji.emojize(':strawberry:'), 
                9 : emoji.emojize(':ticket:')} 

Зададим начальную таблицу выплат произвольным образом в качестве словаря:

In [6]:
pay_dict = {0 : 100,
            1 : 10, 
            2 : 15, 
            3 : 20, 
            4 : 25, 
            5 : 30, 
            6 : 35,
            7 : 40,
            8 : 45} 

Функция подсчета выиграша с линии:

In [7]:
def pay_line(i, symbols, wager=1, display=False):
    message = ''
    line_win = 0
    symbol = symbols[0]
    # если комбинация простых символов выиграла с символов wild, то нужно считать выигрыш по простым символам
    if symbol == 0:
        symbol = symbols[1]
        if symbol == 0:
            symbol = symbols[2]
    # за метч символов фриспинов выплаты нет
    if symbol == 9:
        line_win = 0
        
    else:
        line_win = pay_dict[symbol] * wager
    # вывод на экран
        if display == True:
            message += 'Line '  + str(i) + ' wins ' + str(line_win) + ' credits with ' + meaning_dict[symbol] + " symbol!"
            print(message)
        
    return line_win, symbol

Функция вращения лент:

In [8]:
def spin(reels):
    spin_values = [[0,0,0],[0,0,0],[0,0,0]]
    for i, object in enumerate(reels):
        length = len(reels[i])
        stop_value = random.randint(0, length-1)
        spin_values[1][i] = reels[i][stop_value]
        if stop_value == 0:
            spin_values[0][i] = reels[i][-1]
        else:
            spin_values[0][i] = reels[i][stop_value - 1]
        if stop_value == len(reels[i])-1:
            spin_values[2][i] = reels[i][0]
        else:
            spin_values[2][i] = reels[i][stop_value +1]
    return spin_values

Функция определения выиграшной линии:

In [9]:
def match(spin_values, purse, wager=1, free_spins=False, display=False):
    
    line_1_win = 0
    symbol_l1 = 0
    line_2_win = 0
    symbol_l2 = 0
    line_3_win = 0
    symbol_l3 = 0
    line_4_win = 0
    symbol_l4 = 0
    line_5_win = 0
    symbol_l5 = 0
    total_win = 0
    
    if free_spins == True:
        purse.debit(0)
        if display == True:
            print('free spin game')
    else:
        purse.debit(5 * wager)
        if display == True:
            print('Game started')
            print('Your bet on each line: ', wager)
            print('Total bet', 5 * wager)
            
    if display == True:     
        print()
        print(meaning_dict[spin_values[0][0]], meaning_dict[spin_values[0][1]], meaning_dict[spin_values[0][2]])
        print(meaning_dict[spin_values[1][0]], meaning_dict[spin_values[1][1]], meaning_dict[spin_values[1][2]])
        print(meaning_dict[spin_values[2][0]], meaning_dict[spin_values[2][1]], meaning_dict[spin_values[2][2]])
        print()
    
    # здесь и далее условия задаются в таком порядке:
    # условие равенства символов, причем равенство символов free spins здесь не учитвается 
    # если первый и второй символ wild
    # если второй и третий символ wild
    # если первый и третий символ wild
    # если первый и второй равны, а третий wild
    # если первый и третий равны, а второй wild
    # если второй и третий равны, а первый wild
    
    # Первая линия с координатами (0,0) (0,1) (0,2) 
    if (spin_values[0][0] == spin_values[0][1]  == spin_values[0][2] != 9) or \
       (spin_values[0][0] == spin_values[0][1] == 0 or \
        spin_values[0][1] == spin_values[0][2] == 0 or \
        spin_values[0][0] == spin_values[0][2] == 0 or \
       (spin_values[0][0] == spin_values[0][1]) and spin_values[0][2] == 0 or \
       (spin_values[0][0] == spin_values[0][2]) and spin_values[0][1] == 0 or \
       (spin_values[0][1] == spin_values[0][2]) and spin_values[0][0] == 0): 
            line_1_win, symbol_l1  = pay_line(1, 
                                              [spin_values[0][0], spin_values[0][1], spin_values[0][2]], 
                                              wager, 
                                              display=display)
    # Вторая линия с координатами (1,0) (1,1) (1,2)
    if (spin_values[1][0] == spin_values[1][1]  == spin_values[1][2] != 9) or \
       (spin_values[1][0] == spin_values[1][1] == 0 or \
        spin_values[1][1] == spin_values[1][2] == 0 or \
        spin_values[1][0] == spin_values[1][2] == 0 or \
       (spin_values[1][0] == spin_values[1][1]) and spin_values[1][2] == 0 or \
       (spin_values[1][0] == spin_values[1][2]) and spin_values[1][1] == 0 or \
       (spin_values[1][1] == spin_values[1][2]) and spin_values[1][0] == 0):   
            line_2_win, symbol_l2 = pay_line(2, 
                                             [spin_values[1][0], spin_values[1][1], spin_values[1][2]],
                                             wager,
                                             display=display)
    # Третья линия с координатами (2,0) (2,1) (2,2)
    if (spin_values[2][0] == spin_values[2][1]  == spin_values[2][2] != 9) or \
       (spin_values[2][0] == spin_values[2][1] == 0 or \
        spin_values[2][1] == spin_values[2][2] == 0 or \
        spin_values[2][0] == spin_values[2][2] == 0 or \
       (spin_values[2][0] == spin_values[2][1]) and spin_values[2][2] == 0 or \
       (spin_values[2][0] == spin_values[2][2]) and spin_values[2][1] == 0 or \
       (spin_values[2][1] == spin_values[2][2]) and spin_values[2][0] == 0):  
            line_3_win, symbol_l3 = pay_line(3,
                                            [spin_values[2][0], spin_values[2][1], spin_values[2][2]],
                                            wager, 
                                            display=display)
    # Четвертая линия с координатами (2,0) (1,1) (0,2)
    if (spin_values[2][0] == spin_values[1][1]  == spin_values[0][2] != 9) or \
       (spin_values[2][0] == spin_values[1][1] == 0 or \
        spin_values[1][1] == spin_values[0][2] == 0 or \
        spin_values[2][0] == spin_values[2][2] == 0 or \
       (spin_values[2][0] == spin_values[1][1]) and spin_values[0][2] == 0 or \
       (spin_values[2][0] == spin_values[0][2]) and spin_values[1][1] == 0 or \
       (spin_values[1][1] == spin_values[0][2]) and spin_values[2][0] == 0):  
            line_4_win, symbol_l4 = pay_line(4, 
                                            [spin_values[2][0], spin_values[1][1], spin_values[0][2]],
                                            wager, 
                                            display=display)
    # Пятая линия с координатами (0,0) (1,1) (2,2)
    if (spin_values[0][0] == spin_values[1][1]  == spin_values[2][2] != 9) or \
       (spin_values[0][0] == spin_values[1][1] == 0 or \
        spin_values[1][1] == spin_values[2][2] == 0 or \
        spin_values[0][0] == spin_values[2][2] == 0 or \
       (spin_values[0][0] == spin_values[1][1]) and spin_values[2][2] == 0 or \
       (spin_values[0][0] == spin_values[2][2]) and spin_values[1][1] == 0 or \
       (spin_values[1][1] == spin_values[2][2]) and spin_values[0][0] == 0): 
            line_5_win, symbol_l5 = pay_line(5, 
                                            [spin_values[0][0], spin_values[1][1], spin_values[2][2]],
                                            wager, 
                                            display=display)
        
    total_win = line_1_win + line_2_win + line_3_win + line_4_win + line_5_win
    
    if display == True: 
        print()
        print('total win: ', total_win, 'credits')
    
    purse.credit(total_win)
    
    if display == True: 
        print('balance: ', purse.get_balance())
        print()
        
    return spin_values, wager, total_win, line_1_win, symbol_l1, line_2_win, symbol_l2, line_3_win, symbol_l3, \
           line_4_win, symbol_l4, line_5_win, symbol_l5

*Примечание.* Визуальное расположение линий выиграша представлены на рисунке:

In [10]:
Image(url= 'https://4.downloader.disk.yandex.ru/preview/917810f273f00756f8cac5914a1fc15035eea21c03d5b09c2c6260a98627ce03/inf/MTLRFjM_NIdWUghYFnM1MulxXC0D_n1ycSuRuQfAlWxXr-1OwDpb9KkRoqy3j6vODednV_UmC_r8tIn2m__ngg%3D%3D?uid=952075579&filename=lines.jpg&disposition=inline&hash=&limit=0&content_type=image%2Fjpeg&owner_uid=952075579&tknv=v2&size=3378x1340')

Функция фри-спиновой игры:

In [11]:
def free_spins_game(spin_values, display, purse):
    fs_total_win = 0
    fs_count = 1
    for _ in range(10):
        spin_values, wager, total_win, line_1_win, symbol_l1, line_2_win, symbol_l2, line_3_win, symbol_l3, \
        line_4_win, symbol_l4, line_5_win, symbol_l5 = match(spin(reels_fs), free_spins=True, display=display, purse=purse)
        fs_total_win += total_win
    return fs_total_win, fs_count

Соединим все блоки воедино и создадим функцию старта игры:

In [12]:
def start_game(display, reels, reels_fs, purse, wager=1):
    fs_total_win = 0
    fs_count = 0
    
    # старт игры и подсчет выигрыша
    spin_values, wager, total_win, line_1_win, symbol_l1, line_2_win, symbol_l2, line_3_win, symbol_l3, \
        line_4_win, symbol_l4, line_5_win, symbol_l5 = match(spin(reels), display=display, wager=wager, purse=purse)
    
    # проверка на срабатывания фри-спиновой игры       
    if ((spin_values[0][0] == 9) or (spin_values[1][0] == 9) or (spin_values[2][0] == 9)) and  \
       ((spin_values[0][1] == 9) or (spin_values[1][1] == 9) or (spin_values[2][1] == 9)) and  \
       ((spin_values[0][2] == 9) or (spin_values[1][2] == 9) or (spin_values[2][2] == 9)):
        fs_total_win, fs_count = free_spins_game(spin_values, display=display, purse=purse)
        
    total_win_final = total_win + fs_total_win
    
    
    return total_win_final, wager, fs_total_win, fs_count, line_1_win, symbol_l1, line_2_win, symbol_l2, \
           line_3_win, symbol_l3, line_4_win, symbol_l4, line_5_win, symbol_l5

Зададим простые ленты для теста игры (добавим больше символов, которые соотвествуют фри-спинам для проверки работы фриспиновых игр):

In [13]:
reels = [[0,1,2,9,4,5,9,7,9,2], [0,1,2,9,3,5,6,7,8,9], [0,0,2,3,9,5,6,7,8,2]]

In [14]:
reels_fs = [[0,1,2,3,4,5,6,7,8], [0,1,2,3,4,5,6,7,8], [0,1,2,3,4,5,6,7,8]]

Инициализация кошелька:

In [15]:
purse = Purse(balance=100)

Старт игры:

In [16]:
start_game(display=True, reels=reels, reels_fs=reels_fs, purse=purse, wager=1)

Game started
Your bet on each line:  1
Total bet 5

🍆 🎫 ⭐
🎫 ⭐ ⭐
🍉 🍒 🍇

Line 4 wins 40 credits with 🍉 symbol!

total win:  40 credits
balance:  135



(40, 1, 0, 0, 0, 0, 0, 9, 0, 0, 40, 7, 0, 0)

Игра работает корректно, теперь для настройки баланса игры напишем функцию-тест:

In [367]:
def test_balance(iterations, returns=False, bootstrap=False):
    total_win_final_count = 0
    fs_total_win_count = 0
    fs_counts = 0
    wager_count = 0
    symbol_wins = []
    symbol_stat = []


    for i in range(iterations):
        total_win_final, wager, fs_total_win, fs_count, line_1_win, \
        symbol_l1, line_2_win, symbol_l2, line_3_win, symbol_l3, line_4_win,\
        symbol_l4, line_5_win, symbol_l5 = start_game(display=False, reels=reels, reels_fs=reels_fs, purse=purse, wager=1)
        symbol_wins.append([line_1_win, symbol_l1, line_2_win, symbol_l2, line_3_win, \
                         symbol_l3, line_4_win, symbol_l4, line_5_win, symbol_l5])
        if line_1_win != 0:
            symbol_stat.append([symbol_l1, line_1_win])
        if line_2_win != 0:
            symbol_stat.append([symbol_l2, line_2_win])
        if line_3_win != 0:
            symbol_stat.append([symbol_l3, line_3_win])
        if line_4_win != 0:
            symbol_stat.append([symbol_l4, line_4_win])
        if line_5_win != 0:
            symbol_stat.append([symbol_l5, line_5_win])
     
        total_win_final_count += total_win_final
        fs_total_win_count += fs_total_win
        wager_count += 5 * wager 
        fs_counts += fs_count
        total_payout = total_win_final_count * 100 / wager_count
        fs_payout = fs_total_win_count * 100 / wager_count
        fs_ratio = fs_counts / iterations
        
        
        
        
    symbol_stat_df = pd.DataFrame(symbol_stat, columns=['symbol', 'win'])
    symbol_stat = (symbol_stat_df.groupby(['symbol'])['win'].sum() * 100 / wager_count).values
    
    if bootstrap == False:
        print('total_win: ', total_win_final_count, 'total_wager: ', wager_count,
              'total_win_fs: ', fs_total_win_count, 'fs_counts: ', fs_counts)
        print('---')
        print('total_payout: ', total_payout, '%',
              'fs_payout: ', fs_payout, '%')
        print('fs_ratio: ', fs_ratio)
        print('---')
        print('symbol payouts (without free spins) in percents')
        print((symbol_stat_df.groupby(['symbol'])['win'].sum() * 100 / wager_count))
        
    if returns == True:
        # вывод для бутстрепа
        return total_payout, fs_payout, fs_ratio, symbol_stat  

In [368]:
test_balance(1000)

total_win:  3415 total_wager:  5000 total_win_fs:  1045 fs_counts:  13
---
total_payout:  68.3 % fs_payout:  20.9 %
fs_ratio:  0.013
---
symbol payouts (without free spins) in percents
symbol
1    8.4
2    3.9
3    2.8
4    6.5
5    4.2
6    6.3
7    7.2
8    8.1
Name: win, dtype: float64


Функция теста работает корректно. Приступим к настройке баланса игры.

## Подбор лент для барабанов и получение необходимого payout

Для подбора входных параметров будем производить бутстреп.

Так как в задании ничего не сказано о расположении символов на барабане (напр. нахождение нескольких символов подряд на ленте), зададим количество каждого в ленте и будем перемешивать их с помощью библиотеки `random`.

Задаем начальные (неперемешанные) ленты, исходя из условия, что число символов на ленте должно быть до 200, а символов у нас 10 - возьмем максимальное число символов и распределим их равномерно:

In [19]:
reel_X = 20 * [0] + 20 * [1] + 20 * [2] + 20 * [3] + 20 * [4] + 20 * [5] + 20 * [6] + 20 * [7] + 20 * [8] + 20 * [9]

In [20]:
len(reel_X)

200

In [21]:
reel_Y = 20 * [0] + 20 * [1] + 20 * [2] + 20 * [3] + 20 * [4] + 20 * [5] + 20 * [6] + 20 * [7] + 20 * [8] + 20 * [9]

In [22]:
len(reel_Y)

200

In [23]:
reel_Z = 20 * [0] + 20 * [1] + 20 * [2] + 20 * [3] + 20 * [4] + 20 * [5] + 20 * [6] + 20 * [7] + 20 * [8] + 20 * [9]

In [24]:
len(reel_Z)

200

И фри-спиновые ленты (по 22 символа):

In [25]:
reel_X_fs = 22 * [0] + 22 * [1] + 22 * [2] + 22 * [3] + 22 * [4] + 22 * [5] + 22 * [6] + 22 * [7] + 22 * [8]

In [26]:
len(reel_X_fs)

198

In [27]:
reel_Y_fs = 22 * [0] + 22 * [1] + 22 * [2] + 22 * [3] + 22 * [4] + 22 * [5] + 22 * [6] + 22 * [7] + 22 * [8]

In [28]:
len(reel_Y_fs)

198

In [29]:
reel_Z_fs = 22 * [0] + 22 * [1] + 22 * [2] + 22 * [3] + 22 * [4] + 22 * [5] + 22 * [6] + 22 * [7] + 22 * [8]

In [30]:
len(reel_Z_fs)

198

Перемешаем элементы в лентах:

In [31]:
reel_X_shuffled = random.sample(reel_X, k=len(reel_X))

In [32]:
reel_Y_shuffled = random.sample(reel_Y, k=len(reel_Y))

In [33]:
reel_Z_shuffled = random.sample(reel_Z, k=len(reel_Z))

In [34]:
reel_X_fs_shuffled = random.sample(reel_X_fs, k=len(reel_X_fs))

In [35]:
reel_Y_fs_shuffled = random.sample(reel_Y_fs, k=len(reel_Y_fs))

In [36]:
reel_Z_fs_shuffled = random.sample(reel_Z_fs, k=len(reel_Z_fs))

Объединим ленты:

In [37]:
reels = [reel_X_shuffled, reel_Y_shuffled, reel_Z_shuffled]

In [240]:
reels_fs = [reel_X_fs_shuffled, reel_Y_fs_shuffled, reel_Z_fs_shuffled]

Напишем функцию для бутстрепа:

In [383]:
def bootstrap(samples, iterations):
    values = []
    symbol_values = []
    # tqdm для прогресс-бара
    for _ in tqdm(range(samples)):
        total_payout, fs_payout, fs_ratio, symbol_stat = test_balance(iterations, returns=True, bootstrap=True)
        
        values.append([total_payout, fs_payout, fs_ratio])
        symbol_values.append(symbol_stat)
            
    values = pd.DataFrame(values, columns=['total_payout', 'fs_payout', 'fs_ratio'])
    symbol_values = pd.DataFrame(symbol_values, columns=['symbol_0_payout', 'symbol_1_payout',
                                                         'symbol_2_payout', 'symbol_3_payout',
                                                         'symbol_4_payout', 'symbol_5_payout',
                                                         'symbol_6_payout', 'symbol_7_payout',
                                                         'symbol_8_payout'])
    mean = values.sum() / samples
    symbol_mean = symbol_values.sum() / samples
    print('general params')
    print(mean)
    print('symbol params')
    print(symbol_mean)

Запустим бутстреп (1000 раз для 1000 игр):

In [384]:
symbol_values = bootstrap(1000, 1000)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 109.88it/s]


general params
total_payout    91.524200
fs_payout       34.142100
fs_ratio         0.015192
dtype: float64
symbol params
symbol_0_payout     7.0582
symbol_1_payout     4.1662
symbol_2_payout     4.4862
symbol_3_payout     5.6180
symbol_4_payout     6.9442
symbol_5_payout     8.0850
symbol_6_payout     9.0278
symbol_7_payout    10.4143
symbol_8_payout     1.5822
dtype: float64


Выведем словарь с выплатами:

In [50]:
pay_dict

{0: 100, 1: 10, 2: 15, 3: 20, 4: 25, 5: 30, 6: 35, 7: 40, 8: 45}

Очевидно, что необходимо снизить число символов **Wild**, а так же немного снизить пейаут фри-спиновой игры. Из открытого источника: пейаут фриспиновой игры должен составлять 30-40% от общего пейаута [ссылка] URL: https://www.youtube.com/watch?v=Or5fOyMLju0&t=1001s

Изменим число символов на лентах:
- число символов **Wild** снизим до 10, 12 и 8 на каждой ленте соответственно;
- число первых обычных символов увеличим до 33, 32 и 34 соответственно;
- число символов **Free Spins** снизим до 19, 18 и 17 соответственно.

In [158]:
reel_X = 10 * [0] + 33 * [1] + 20 * [2] + 20 * [3] + 20 * [4] + 20 * [5] + 20 * [6] + 20 * [7] + 20 * [8] + 19 * [9]

In [159]:
len(reel_X)

202

In [160]:
reel_Y = 12 * [0] + 32 * [1] + 20 * [2] + 20 * [3] + 20 * [4] + 20 * [5] + 20 * [6] + 20 * [7] + 20 * [8] + 18 * [9]

In [161]:
len(reel_Y)

202

In [162]:
reel_Z = 8 * [0] + 34 * [1] + 20 * [2] + 20 * [3] + 20 * [4] + 20 * [5] + 20 * [6] + 20 * [7] + 20 * [8] + 17 * [9]

In [163]:
len(reel_Z)

199

In [164]:
len(reel_Z_fs)

198

Перемешаем элементы в лентах:

In [165]:
reel_X_shuffled = random.sample(reel_X, k=len(reel_X))

In [166]:
reel_Y_shuffled = random.sample(reel_Y, k=len(reel_Y))

In [167]:
reel_Z_shuffled = random.sample(reel_Z, k=len(reel_Z))

Объединим ленты:

In [239]:
reels = [reel_X_shuffled, reel_Y_shuffled, reel_Z_shuffled]

Запустим бутстреп:

In [169]:
bootstrap(1000, 1000)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 109.77it/s]

general params
total_payout    112.868600
fs_payout        35.110200
fs_ratio          0.015707
dtype: float64
symbol params
symbol_0_payout     6.667200
symbol_1_payout     6.882100
symbol_2_payout     5.747700
symbol_3_payout     7.315100
symbol_4_payout     8.863000
symbol_5_payout    10.374000
symbol_6_payout    12.200500
symbol_7_payout    13.508700
symbol_8_payout    14.622877
dtype: float64


Еще снизим число символов **Wild** до 6, 7 и 6 соответственно:

In [182]:
reel_X = 6 * [0] + 33 * [1] + 20 * [2] + 20 * [3] + 20 * [4] + 20 * [5] + 20 * [6] + 20 * [7] + 20 * [8] + 19 * [9]

In [183]:
len(reel_X)

198

In [184]:
reel_Y = 7 * [0] + 32 * [1] + 20 * [2] + 20 * [3] + 20 * [4] + 20 * [5] + 20 * [6] + 20 * [7] + 20 * [8] + 18 * [9]

In [185]:
len(reel_Y)

197

In [186]:
reel_Z = 6 * [0] + 34 * [1] + 20 * [2] + 20 * [3] + 20 * [4] + 20 * [5] + 20 * [6] + 20 * [7] + 20 * [8] + 17 * [9]

In [187]:
len(reel_Z)

197

In [188]:
len(reel_Z_fs)

198

Перемешаем элементы в лентах:

In [200]:
reel_X_shuffled = random.sample(reel_X, k=len(reel_X))

In [201]:
reel_Y_shuffled = random.sample(reel_Y, k=len(reel_Y))

In [202]:
reel_Z_shuffled = random.sample(reel_Z, k=len(reel_Z))

Объединим ленты:

In [203]:
reels = [reel_X_shuffled, reel_Y_shuffled, reel_Z_shuffled]

Запустим более тяжелый бутстреп (10000 раз по 10000 игр): 

In [386]:
bootstrap(10000, 10000)

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [14:15<00:00, 11.69it/s]


general params
total_payout    91.397264
fs_payout       34.062283
fs_ratio         0.015184
dtype: float64
symbol params
symbol_0_payout    1.926658
symbol_1_payout    6.990393
symbol_2_payout    3.752968
symbol_3_payout    4.926095
symbol_4_payout    6.083742
symbol_5_payout    7.252033
symbol_6_payout    8.409508
symbol_7_payout    9.595252
symbol_8_payout    8.398332
dtype: float64


Отлично, в нужный **payout** мы попали и его доля пейатуа от фри-спиновых игр тоже хорошая. Характеристики слот-машины подтверждены. Задачу можно считать решенной.

Запишем ленты в датафреймы:

In [216]:
reels_df = pd.DataFrame(list(map(list, zip(*reels))), columns=['reel_X', 'reel_Y', 'reel_Z'])
reels_fs_df = pd.DataFrame(list(map(list, zip(*reels_fs))), columns=['reel_X_fs', 'reel_Y_fs', 'reel_Z_fs'])

Сохраним ленты на диск:

In [230]:
reels_df.to_excel('D:/reels_df.xlsx')
reels_fs_df.to_excel('D:/reels_fs_df.xlsx')

## Ответы на поставленные вопросы

- При решении задачи использовались три сплошные линии и две диагональные линии (см. рис. 1);
- Ленты (барабаны) слота представлены в приложеном к тетрадке файле `Excel`;
- Таблица выплат и payout каждой комбинации

|Символ |Выплата, credits|Payout, %|
|-------|-------|------|
| ⭐ (**Wild**)|100| 1.93
|🍒 (**High 8**)| 10| 6.99
|🍇(**High 7**)| 15| 3.75
|🍌(**High 6**)|20| 4.93
|🍏(**High 5**)|25| 6.08
|🍆(**High 4**)|30| 7.25
|🍍(**High 3**)|35| 8.41
|🍉(**High 2**)|40| 9.59
|🍓(**High 1**)|45| 8.39
|🎫(**Free Spins 1**)|-|34.06

При этом общий пейаут (**total payout**) : 91.40 %, доля **Free Spins** : 0.015